In [9]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import norm
from astropy.table import Table
from astropy.io import ascii

from sklearn.mixture import GMM

from astroML.datasets import fetch_sdss_sspp
from astroML.decorators import pickle_results
from astroML.plotting.tools import draw_ellipse

In [10]:
from astroML.plotting import setup_text_plots
setup_text_plots(fontsize=8, usetex=True)

In [11]:
data = Table.read('breddkatalog_fluxes.csv', converters = {'col1': [ascii.convert_numpy(np.str)]})

In [12]:
X = np.vstack([data['alpha'], data['beta']]).T

In [13]:
X = X[::5]

In [14]:
N = np.arange(1, 14)

@pickle_results("GMM_GRB.pkl")
def compute_GMM(N, covariance_type='full', n_iter=1000):
    models = [None for n in N]
    for i in range(len(N)):
        print N[i]
        models[i] = GMM(n_components=N[i], n_iter=n_iter,
                        covariance_type=covariance_type)
        models[i].fit(X)
    return models

models = compute_GMM(N)

AIC = [m.aic(X) for m in models]
BIC = [m.bic(X) for m in models]

i_best = np.argmin(BIC)
gmm_best = models[i_best]
print "best fit converged:", gmm_best.converged_
print "BIC: n_components =  %i" % N[i_best]

@pickle_results: using precomputed results from 'GMM_GRB.pkl'
best fit converged: True
BIC: n_components =  6


In [15]:
# compute 2D density
alpha_bins = 51
beta_bins = 51
H, alpha_bins, beta_bins = np.histogram2d(data['alpha'], data['beta'],
                                          (alpha_bins, beta_bins))

Xgrid = np.array(map(np.ravel,
                     np.meshgrid(0.5 * (alpha_bins[:-1]
                                        + alpha_bins[1:]),
                                 0.5 * (beta_bins[:-1]
                                        + beta_bins[1:])))).T
log_dens = gmm_best.score(Xgrid).reshape((51, 51))

In [ ]:
# Plot the results
fig = plt.figure(figsize=(5, 1.66))
fig.subplots_adjust(wspace=0.45,
                    bottom=0.25, top=0.9,
                    left=0.1, right=0.97)

# plot density
ax = fig.add_subplot(131)
ax.imshow(H.T, origin='lower', interpolation='nearest', aspect='auto',
          extent=[alpha_bins[0], alpha_bins[-1],
                  beta_bins[0], beta_bins[-1]],
          cmap=plt.cm.binary)
ax.set_xlabel(r'$\rm [alpha]$')
ax.set_ylabel(r'$\rm [beta]$')
ax.xaxis.set_major_locator(plt.MultipleLocator(0.3))
ax.set_xlim(-1.101, 0.101)
ax.text(0.93, 0.93, "Input",
        va='top', ha='right', transform=ax.transAxes)

# plot AIC/BIC
ax = fig.add_subplot(132)
ax.plot(N, AIC, '-k', label='AIC')
ax.plot(N, BIC, ':k', label='BIC')
ax.legend(loc=1)
ax.set_xlabel('N components')
plt.setp(ax.get_yticklabels(), fontsize=7)

# plot best configurations for AIC and BIC
ax = fig.add_subplot(133)
ax.imshow(np.exp(log_dens),
          origin='lower', interpolation='nearest', aspect='auto',
          extent=[alpha_bins[0], alpha_bins[-1],
                  beta_bins[0], beta_bins[-1]],
          cmap=plt.cm.binary)

ax.scatter(gmm_best.means_[:, 0], gmm_best.means_[:, 1], c='w')
for mu, C, w in zip(gmm_best.means_, gmm_best.covars_, gmm_best.weights_):
    draw_ellipse(mu, C, scales=[1.5], ax=ax, fc='none', ec='k')

ax.text(0.93, 0.93, "Converged",
        va='top', ha='right', transform=ax.transAxes)

ax.set_xlim(-4, 8)
ax.set_ylim(beta_bins[0], beta_bins[-1])
ax.xaxis.set_major_locator(plt.MultipleLocator(0.3))
ax.set_xlabel(r'$\rm [alpha]$')
ax.set_ylabel(r'$\rm [beta]$')

plt.show()